In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model


In [3]:
# Check if GPU is available
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("GPU not found. Please check your Kaggle settings.")

Default GPU Device: /device:GPU:0


In [4]:
df = pd.read_csv("/kaggle/input/block-1024-snr2/merged_data_block1024_snr2.csv")

In [46]:
testdf = pd.read_csv("/kaggle/input/testdata-69/test.csv")

In [5]:
df.shape

(405000, 2)

In [6]:
feed = pd.DataFrame()
target = pd.DataFrame()

In [7]:
feed['encoded_data_string'] = df['encoded_data_string']
target['encoding'] = df['encoding']

In [8]:
X = feed  # Features (all columns in the DataFrame)
y = target  # Target variable (if applicable)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
data = []
for bit_string in X_train['encoded_data_string']:
    bit_array = np.array(list(bit_string), dtype=int)
    data.append(bit_array)

In [11]:
Xtrain_array = np.reshape(data, (324000, 1024, 1))

In [12]:
Xtrain_array.shape

(324000, 1024, 1)

In [13]:
data1 = []
for bit_string in X_test['encoded_data_string']:
    bit_array = np.array(list(bit_string), dtype=int)
    data1.append(bit_array)

In [48]:
data2 = []
for bit_string in testdf:
    bit_array1 = np.array(list(bit_string), dtype=int)
    data2.append(bit_array1)

In [49]:
data2

[array([0, 1, 0, ..., 1, 0, 1])]

In [47]:
testdf

,0100101010001101100010111111010110101001011001110110001010011011100010111001000101011001011110101110011110101000001100010110100000100100110101000010101010100011010110101000000110100110100000010111100001101011001101001001110111000100100011101001110111100001010100111111110101011010110000111110011100010010010110100010101110111010100110111100000101000110100101001000000010010001010000000100001100101100101111001001011010111000101110100001101000111101010001101100011010110001000110000110001011101110110010101001111111011010111010000011010000011100111110101010010101111011001011110111010100111010100110000101111010110100001100101000011000101111000000101110011011100110111000110011110001101001110001101010110010110011100111010001110111000010000101101111101000010001101000011101011110011010111010101000010010000001111010001110010101100111000001110001000100101110010110101000100001110100000011010010010101011100000110111001101000110011111001000100011000011011101110010100101101100110011111010000101110101010011100011010000110100101


In [14]:
Xtest_array = np.reshape(data1, (81000, 1024, 1))

In [15]:
ytrain_array = y_train['encoding'].values
ytest_array = y_test['encoding'].values

In [16]:
# np.save('Xtrain_array.txt', Xtrain_array)

In [17]:
ytrain_mapped = ytrain_array - 1

In [18]:
np.unique(ytrain_array)

array([1, 2, 3, 4])

In [19]:
# np.save('ytrain_array.txt', ytrain_array)

In [20]:
# np.save('Xtest_array.txt', Xtest_array)

In [21]:
# np.save('ytest_array.txt', ytest_array)

In [22]:
Xtrain_array_gpu = tf.convert_to_tensor(Xtrain_array, dtype=tf.float32)
ytrain_mapped_gpu = tf.convert_to_tensor(ytrain_mapped, dtype=tf.int32)

In [51]:
Xtrain_array_gpu1 = tf.convert_to_tensor(data2, dtype=tf.float32)


In [24]:
input_shape = (1024, 1)

# Assuming Xtrain_array_gpu and ytrain_mapped_gpu are TensorFlow tensors
train_dataset = tf.data.Dataset.from_tensor_slices((Xtrain_array_gpu, ytrain_mapped_gpu)).shuffle(buffer_size=2000).batch(200)
test_dataset = tf.data.Dataset.from_tensor_slices((Xtest_array, y_test)).batch(200)


In [25]:
# Define the model within the strategy scope
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


In [26]:
with strategy.scope():
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [31]:
model.load_weights('/kaggle/input/weight97a6e/current_model_weights97a6e.h5')

In [32]:
loaded_model = load_model('/kaggle/input/model-saved-2048-s2-97a/sih_2048_snr2_model')

In [28]:
# Add ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(filepath='model_checkpoint.h5', save_best_only=True, period=3)# Train the model using the distributed dataset
early_stopping = EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.0002, patience=3, min_lr=1e-6)



In [33]:
model.fit(train_dataset, epochs=200, validation_data=test_dataset,batch_size=200,callbacks=[model_checkpoint,early_stopping,reduce_lr])

Epoch 1/200
1620/1620 [==============================] - 622s 384ms/step - loss: 0.2454 - accuracy: 0.9077 - val_loss: nan - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/200
1620/1620 [==============================] - 617s 381ms/step - loss: 0.2014 - accuracy: 0.9168 - val_loss: nan - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/200
1620/1620 [==============================] - 617s 381ms/step - loss: 0.1999 - accuracy: 0.9160 - val_loss: nan - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/200
1620/1620 [==============================] - 616s 381ms/step - loss: 0.1859 - accuracy: 0.9206 - val_loss: nan - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/200
1620/1620 [==============================] - 616s 380ms/step - loss: 0.1808 - accuracy: 0.9214 - val_loss: nan - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 6/200
1620/1620 [==============================] - 616s 381ms/step - loss: 0.1742 - accuracy: 0.9229 - val_loss: nan - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 7/200
1620/1620 

KeyboardInterrupt: 

In [34]:
model.save_weights('current_model_weights99_1024.h5')

In [54]:
predictions=model.predict(Xtrain_array_gpu1)

CancelledError: Graph execution error:

RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/Identity_2/_57}}]] [Op:__inference_predict_function_429915]

In [35]:
np.unique(ytest_array)

array([1, 2, 3, 4])

In [36]:
ytest_mapped = ytest_array - 1

In [37]:
np.unique(ytest_mapped)

array([0, 1, 2, 3])

In [38]:
ytest_mapped

array([1, 0, 0, ..., 2, 2, 0])

In [39]:

if np.isnan(ytest_mapped).any() or np.isinf(ytest_mapped).any():
    print("True labels contain NaN or infinite values.")
    
else:
    print("np")


np


In [40]:
ytest_mapped.shape

(81000,)

In [41]:
ytest_array.shape

(81000,)

In [42]:
Xtest_array_gpu = tf.convert_to_tensor(Xtest_array, dtype=tf.float32)
ytest_mapped_gpu = tf.convert_to_tensor(ytest_mapped, dtype=tf.int32)

In [43]:
loss, accuracy = model.evaluate(Xtest_array_gpu, ytest_mapped_gpu)

print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy * 100:.2f}%")

2532/2532 [==============================] - 200s 77ms/step - loss: 0.0099 - accuracy: 0.9970
Loss: 0.0099
Accuracy: 99.70%


In [44]:
model.save('/kaggle/working/sih_1024_snr2_model')

In [39]:
# Save the model as a pickle file
with open('sih_2048_snr2.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)